In [4]:
import os
os.chdir("C:\\Users\\rivil\\PycharmProjects\\projects\\DeepCNNClassifier\\")
os.getcwd()

'C:\\Users\\rivil\\PycharmProjects\\projects\\DeepCNNClassifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path
#from pickletools import float8

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                create_directories([self.config.artifacts_root])

    def get_prepare_Callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks   # all the info is inside this variable
                                                 # as per name in config.yaml
        
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([Path(config.tensorboard_root_log_dir),Path(model_ckpt_dir)]) # creates 2 directories
       

        prepare_Callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath))

        return prepare_Callbacks_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallbacks:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
                            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir="tb_running_log_dir")

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
                            filepath=self.config.checkpoint_model_filepath,
                            save_best_only=True)

    def get_tb_ckpt_callbacks(self):
        return[ self._create_tb_callbacks,
                self._create_ckpt_callbacks]

    

In [14]:
try:
    config = ConfigurationManager()
    prepare_Callbacks_config = config.get_prepare_Callbacks_config()  
    prepare_Callbacks_model = PrepareCallbacks(config=prepare_Callbacks_config) 
    prepare_Callbacks_model.get_tb_ckpt_callbacks() 

    
except Exception as e:
    raise e
